<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Google-Cloud-Speech-to-Text-API-Streaming-example" data-toc-modified-id="Google-Cloud-Speech-to-Text-API-Streaming-example-1">Google Cloud Speech-to-Text API Streaming example</a></span></li></ul></div>

# Google Cloud Speech-to-Text API Streaming example

- 1분 이상의 긴 오디오 파일을 Streaming 방식으로 처리하는 샘플
- https://cloud.google.com/speech-to-text/docs/streaming-recognize
- google.cloud.speech.SpeechClient().**streaming_recognize(streaming_config, requests)**
- **Audio file size limit : 10M bytes** => 10M 이상의 파일을 처리하기 위해 wave file을 10M 이하 크기로 잘라서 변환하도록 개선

In [1]:
import pyaudio
import wave
import os
import io
import csv

import numpy as np

from nltk.tokenize import sent_tokenize, word_tokenize

# for speaker_diarization (use speech_v1p1beta1)
from google.cloud import speech_v1p1beta1 as speech
from google.cloud.speech_v1p1beta1 import enums
from google.cloud.speech_v1p1beta1 import types

In [2]:
def transcribe_streaming(stream_file):
    """Streams transcription of the given audio file."""
    words_with_tag = []
    transcripts = []

    client = speech.SpeechClient()

    with io.open(stream_file, 'rb') as audio_file:
        content = audio_file.read()

    # In practice, stream should be a generator yielding chunks of audio data.
    stream = [content]

    requests = (types.StreamingRecognizeRequest(audio_content=chunk)
                for chunk in stream)

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='ko-KR',
        enable_automatic_punctuation=True,
        enable_word_time_offsets=True,
        enable_speaker_diarization=True,    # 한국어 지원 안됨 (speaker_tag가 모두 동일인으로 분류됨)
        diarization_speaker_count=3)

    streaming_config = types.StreamingRecognitionConfig(config=config)
    
    # streaming_recognize returns a generator.
    responses = client.streaming_recognize(streaming_config, requests)
    
    for response in responses:
        for result in response.results:
            alternatives = result.alternatives
            for alternative in alternatives:
                print(u'Transcript: {}'.format(alternative.transcript))
                transcripts.append(alternative.transcript)    # punctuation 포함된 문장을 사용하기 위해 저장
                for words in alternative.words:
                    word = words.word
                    start_time = round(words.start_time.seconds + words.start_time.nanos * 1e-9, 3)
                    end_time = round(words.end_time.seconds + words.end_time.nanos * 1e-9, 3)
                    speaker_tag = words.speaker_tag
                    words_with_tag.append([word, start_time, end_time, speaker_tag])    # [word, start_time, end_time, speaker_tag]
        print()
    return words_with_tag, transcripts

In [3]:
def write_wave_frames(frames):
        wave_stream = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        wave_stream.setnchannels(CHANNELS)
        wave_stream.setsampwidth(FORMAT)
        wave_stream.setframerate(RATE)
        wave_stream.writeframes(b''.join(frames))
        wave_stream.close()

In [4]:
# instantiate PyAudio
audio = pyaudio.PyAudio()

In [5]:
# open wave file
FILE_NAME = "경제청문회_4명인터뷰_YTN_16000"
wf = wave.open("audio_file/"+ FILE_NAME + ".wav", 'rb')

In [6]:
# initialize variables
CHUNK = 1024   # 1024 bytes of data read from the buffer
FRAME_SIZE = CHUNK * 4
WAVE_OUTPUT_FILENAME = "wave_output.wav"    # temperary wave file for streaming 
FORMAT=audio.get_sample_size(wf.getsampwidth())
CHANNELS=wf.getnchannels()
RATE=wf.getframerate()
words_with_tags = []
transcripts_arr = []
frames = []
f_count = 0

In [7]:
# read data (first frames)
data = wf.readframes(CHUNK)

# transcribe stream  : wave file을 10M 이하로 잘라서, google STT API에 stream 방식으로 요청
while len(data) > 0:
    frames.append(data)
    f_count += 1
    if f_count == FRAME_SIZE:
        # write frames to wave file for frames
        write_wave_frames(frames)
        
        # transcribe by streaming
        words_with_tag, transcripts = transcribe_streaming(WAVE_OUTPUT_FILENAME)    # frames를 직접 전달 시 google에서 grpc.RpcError 발생하면 cancel 됨 
        words_with_tags.append(words_with_tag)
        transcripts_arr.append(transcripts)
        
        # frame initialize
        frames = []
        f_count = 0
    
    # read data (continue)
    data = wf.readframes(CHUNK)

# transcribe for last frames
write_wave_frames(frames)

# last frames
words_with_tag, transcripts = transcribe_streaming(WAVE_OUTPUT_FILENAME)
words_with_tags.append(words_with_tag)
transcripts_arr.append(transcripts)

Transcript: 한국 땅이 내 건 경제 청문회 여야 각 정당의 반응이 엇갈립니다. 민주당과 바른미래당 정의당은 한국 땅이 업체를 부린다는 입장인 반면에 한국당과 민주평화당 민주당이 청문회를 받아야 한다고 압박합니다.

Transcript:  여론은 반대가 55%로 찬성 31%를 크게 앞지른 것으로 조사됐습니다. 반대 여론이 높은 경제청 문제를 내 건 한국 땅이 문제가 앞으로 정당 지지도 그러면 어떤 영향을 미칠지 궁금합니다. 숫자이동지도 열어. 내서 YTN 더 뉴스가의 실시한 여론조사 결과 분석하겠습니다. 이택수 리얼미터 대표 박지영 윈지코리아컨설팅 부대표 타기 하셨습니다 하십니까? 안녕하세요. 안녕하세요. 경제 청문회 관련한 조사결과부터 다시 한번 정리를 하고 얘기를 이어가겠습니다. 함께 보시죠. 다 한국 땅이 요구한 경찰청 운세 응답자의 55.4%가 반대하는 것으로 나타났습니다. 찬성은 30.9% 였고요. 모른다거나 응답하지않음. 비율은 13.7% 뛰었습니다. 응답자를 지지정당 별로 나눠서 살펴보겠습니다. 민주당 짐승의 결제 또 가도 높았습니다. 응답자중 민주당 지지표의 85.9

Transcript:  퍼센트는 반대했고 한국땅을 지지하는 응답자는 68.2%가 찬성하는 것으로 나타났습니다. 정치 성향 별로 나눠서 조사한 결과에서도 진보측의 반대 여론이 보수층의 찬성 여름보다 경고했습니다. 봉투 측에서의 반대 여론이 35.8%로 경제 청문회를 정치공세로 보내 시각도 포스팅에서도 적지 않아 중고책 전체 여론 부도와 비슷했습니다. 정치 성향 결론은 중도층이 전체 여론과 비슷해 따면 지역별로 나는 수도권과 충청권을 그랬습니다. 결과를 좀 봤습니다. 찬송 30.9% 돼. 반대 55.4% 대표님, 한국 땅에 이런 여러 오늘 예상 했을까요? 예상하지 못했을 것 같습니다. 일단 뭐 큰 틀에서의 결과는 55.4 돼. 30.9 예상가 크게 다르지 않은 듯 세부적으로 보면 이런 감정 보수층 일하는 중이라

Transcript:  건지 아니면 한국당 지지율 제외하고는 대부

In [8]:
# close PyAudio
audio.terminate()

In [9]:
corpus = ""
for transcripts in transcripts_arr:
    for transcript in transcripts:
        corpus += transcript + " "
# print(corpus)

In [10]:
sentence = sent_tokenize(corpus)    # 문장단위 분리
sentence

['한국 땅이 내 건 경제 청문회 여야 각 정당의 반응이 엇갈립니다.',
 '민주당과 바른미래당 정의당은 한국 땅이 업체를 부린다는 입장인 반면에 한국당과 민주평화당 민주당이 청문회를 받아야 한다고 압박합니다.',
 '여론은 반대가 55%로 찬성 31%를 크게 앞지른 것으로 조사됐습니다.',
 '반대 여론이 높은 경제청 문제를 내 건 한국 땅이 문제가 앞으로 정당 지지도 그러면 어떤 영향을 미칠지 궁금합니다.',
 '숫자이동지도 열어.',
 '내서 YTN 더 뉴스가의 실시한 여론조사 결과 분석하겠습니다.',
 '이택수 리얼미터 대표 박지영 윈지코리아컨설팅 부대표 타기 하셨습니다 하십니까?',
 '안녕하세요.',
 '안녕하세요.',
 '경제 청문회 관련한 조사결과부터 다시 한번 정리를 하고 얘기를 이어가겠습니다.',
 '함께 보시죠.',
 '다 한국 땅이 요구한 경찰청 운세 응답자의 55.4%가 반대하는 것으로 나타났습니다.',
 '찬성은 30.9% 였고요.',
 '모른다거나 응답하지않음.',
 '비율은 13.7% 뛰었습니다.',
 '응답자를 지지정당 별로 나눠서 살펴보겠습니다.',
 '민주당 짐승의 결제 또 가도 높았습니다.',
 '응답자중 민주당 지지표의 85.9  퍼센트는 반대했고 한국땅을 지지하는 응답자는 68.2%가 찬성하는 것으로 나타났습니다.',
 '정치 성향 별로 나눠서 조사한 결과에서도 진보측의 반대 여론이 보수층의 찬성 여름보다 경고했습니다.',
 '봉투 측에서의 반대 여론이 35.8%로 경제 청문회를 정치공세로 보내 시각도 포스팅에서도 적지 않아 중고책 전체 여론 부도와 비슷했습니다.',
 '정치 성향 결론은 중도층이 전체 여론과 비슷해 따면 지역별로 나는 수도권과 충청권을 그랬습니다.',
 '결과를 좀 봤습니다.',
 '찬송 30.9% 돼.',
 '반대 55.4% 대표님, 한국 땅에 이런 여러 오늘 예상 했을까요?',
 '예상하지 못했을 것 같습니다.',
 '일단 뭐 큰 틀에서의 결과는 55.4 돼.',
 '30.9 예상가 크게 다르지 않은

In [11]:
# 스크립트 만 Text 파일로 저장 (with newline)
text_file = "text/" + FILE_NAME + ".txt"
with open(text_file, 'w', encoding='utf-8') as f:
    for sent in sentence:
        f.write(sent + "\n")

In [12]:
# print(words_with_tags)
words_array = [wt for wts in words_with_tags for wt in wts]
words_array

[['한국', 0.3, 0.9, 1],
 ['땅이', 0.9, 1.2, 1],
 ['내', 1.2, 1.3, 1],
 ['건', 1.3, 1.6, 1],
 ['경제', 1.6, 2.2, 1],
 ['청문회', 2.2, 2.7, 1],
 ['여야', 2.7, 3.3, 1],
 ['각', 3.3, 3.5, 1],
 ['정당의', 3.5, 4.0, 1],
 ['반응이', 4.0, 4.4, 1],
 ['엇갈립니다', 4.4, 4.8, 1],
 ['민주당과', 4.8, 5.7, 1],
 ['바른미래당', 5.7, 6.3, 1],
 ['정의당은', 6.3, 7.0, 1],
 ['한국', 7.0, 7.6, 1],
 ['땅이', 7.6, 7.9, 1],
 ['업체를', 7.9, 8.4, 1],
 ['부린다는', 8.4, 8.8, 1],
 ['입장인', 8.8, 9.2, 1],
 ['반면', 9.2, 9.3, 1],
 ['에', 9.3, 9.6, 1],
 ['한국당과', 9.6, 10.9, 1],
 ['민주평화당', 10.9, 11.7, 1],
 ['민주당이', 11.7, 12.8, 1],
 ['청문회를', 12.8, 13.5, 1],
 ['받아야', 13.5, 14.2, 1],
 ['한다고', 14.2, 14.5, 1],
 ['압박합니다', 14.5, 15.1, 1],
 ['한국', 0.3, 0.9, 1],
 ['땅이', 0.9, 1.2, 1],
 ['내', 1.2, 1.3, 1],
 ['건', 1.3, 1.6, 1],
 ['경제', 1.6, 2.2, 1],
 ['청문회', 2.2, 2.7, 1],
 ['여야', 2.7, 3.3, 1],
 ['각', 3.3, 3.5, 1],
 ['정당의', 3.5, 4.0, 1],
 ['반응이', 4.0, 4.4, 1],
 ['엇갈립니다', 4.4, 4.8, 1],
 ['민주당과', 4.8, 5.7, 1],
 ['바른미래당', 5.7, 6.3, 1],
 ['정의당은', 6.3, 7.0, 1],
 ['한국', 7.0, 7.6, 1],
 ['땅

In [13]:
# CSV 파일로 저장
csv_file = "text/" + FILE_NAME + ".csv"
with open(csv_file, 'w', encoding='utf-8', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(["word", "start_time", "end_time", "speaker_tag"])
    for words in words_array:
        wr.writerow(words)